### Ready simulation data for NeuroScope

In [ ]:
#this notebook is to expediate the process between simulating the data I need and getting results
#hopefuly I just have to run this notebook and it will all do itself

In [ ]:
import strax 
import straxen 
import numpy as np
import matplotlib.pyplot as plt
import cutax
import pandas as pd
import wfsim

In [ ]:
### Define variables
len_of_SE = 0
len_of_ArS1 = 0
zeros = 0

In [ ]:
se_data = np.load('./sim_data/sim_single_E_id2_peaks.npz')['arr_0']
se_truth = np.load('./sim_data/sim_single_E_id2_truth.npz')['arr_0']
ar_data = np.load('./sim_data/sim_Ar37_id2_peaks.npz')['arr_0']
ar_truth = np.load('./sim_data/sim_Ar37_id2_truth.npz')['arr_0']

In [ ]:
se_pb = np.load('./sim_data/sim_single_E_id2_peak_basics.npz')['arr_0']
ar_pb = np.load('./sim_data/sim_Ar37_id2_peak_basics.npz')['arr_0']

In [ ]:
rn_data = np.load('./saved_data/Rn_run24072_s1_peaks.npz')['arr_0']
rn_pb = np.load('./saved_data/Rn_run24072_s1_peak_basics.npz')['arr_0']

In [ ]:
def dump_matched_peaks(truth, peaks, peak_basics):#, int_type):
    """
    Dump matched peaks in a human readable format
    param truth: truth array to be matched
    param peaks: peaks array to be matched
    return data: np.array with the matched peaks and true info
    """
    #  This is for single electrons
    #if int_type == 1:
    #    truth = truth[truth['n_electron'] == 1]

    #  for now focus on S2
    s2_truth = truth
    s2_peaks = peaks
    s2_peak_basics = peak_basics

    #  This is first attemp to match peaks and true info
    mask = np.zeros(len(s2_peak_basics), dtype=np.bool)
    for i, t in enumerate(s2_truth):
        mask += ((s2_peak_basics['time'] < t['t_mean_photon']) &
                 (s2_peak_basics['endtime'] > t['t_mean_photon']))

    s2_matched = s2_peaks[mask]
    s2_peak_basics = s2_peak_basics[mask]

    #   Do not ask me why... sometimes life is hard...
    if len(s2_matched['time']) != len(s2_truth['time']):
        s2_matched, s2_peak_basics, s2_truth = match_events(s2_matched, s2_peak_basics, s2_truth)

    return s2_matched, s2_peak_basics, s2_truth

In [ ]:
def match_events(s2_matched, s2_peak_basics, s2_truth):
    """
    Match s2 peaks with s2 true info
    param s2_matched: s2 peak array
    param s2_peak_basics: s2 peak_basic array
    param s2_truth: s2 truth array
    param return: s2_ms2_matched, s2_peak_basics, s2_truth matched arrays
    """
    matched = []
    matched_t = []
    '''
    for i, p in enumerate(s2_peak_basics):
        for j, t in enumerate(s2_truth):
            if abs(p['center_time']-t['t_mean_photon']) < 200:  # 200 ns window
                matched.append(i)
                matched_t.append(j)
                break
    '''
                
    for j, t in enumerate(s2_truth):     
        if np.min(abs(t['t_mean_photon'] - s2_peak_basics['center_time'])) < 200:
            pb_loc = np.where(abs(s2_peak_basics['center_time'] - t['t_mean_photon']) == np.min(abs(t['t_mean_photon'] - s2_peak_basics['center_time'])))
            matched.append(pb_loc)
            matched_t.append(j)

    s2_matched = np.take(s2_matched, matched)
    s2_peak_basics = np.take(s2_peak_basics, matched)
    s2_truth = np.take(s2_truth, matched_t)

    return s2_matched, s2_peak_basics, s2_truth

In [ ]:
def compute_wf_and_quantiles(peaks: np.ndarray, bayes_n_nodes: int):
    """
    Compute waveforms and quantiles for a given number of nodes(atributes)
    :param peaks:
    :param bayes_n_nodes: number of nodes or atributes
    :return: waveforms and quantiles
    """
    waveforms = np.zeros((len(peaks), bayes_n_nodes))
    quantiles = np.zeros((len(peaks), bayes_n_nodes))

    num_samples = peaks['data'].shape[1]
    #modified line, original num_samples = peaks['data'].shape[1] 
    step_size = int(num_samples/bayes_n_nodes)
    steps = np.arange(0, num_samples+1, step_size)

    data = peaks['data'].copy() #data = peaks['data'].copy() 
    data[data < 0.0] = 0.0
    for i, p in enumerate(peaks):
        sample_number = np.arange(0, num_samples+1, 1)*p['dt']
        frac_of_cumsum = np.append([0.0], np.cumsum(data[i, :]) / np.sum(data[i, :]))
        cumsum_steps = np.interp(np.linspace(0., 1., bayes_n_nodes, endpoint=False), frac_of_cumsum, sample_number)
        cumsum_steps = np.append(cumsum_steps, sample_number[-1])
        quantiles[i, :] = cumsum_steps[1:] - cumsum_steps[:-1]

    for j in range(bayes_n_nodes):
        waveforms[:, j] = np.sum(data[:, steps[j]:steps[j+1]], axis=1)
    waveforms = waveforms/(peaks['dt']*step_size)[:, np.newaxis]

    del data
    return waveforms, quantiles

In [ ]:
def generate_incl_file_4(filename, xcube, ycube, class_nums):
    '''generates a file that tells NS what the label of each data point is. this needs:
    filename: such as 'data.incl'
    xcube: horizontal dimension of the data cube
    ycube: vertical dimension of the data cube
    class_nums: vector of 3 dimensions detailing where the labels should swtich, the values of each vector should be its own puls the previous'''
    f = open(filename, "w+")
    count = 1
    for i in np.arange(ycube):
        for j in np.arange(xcube):
            if count <= class_nums[0]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' A' + ' \n')
            if count > class_nums[0] and count <= class_nums[1]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' H' + '\n')
            if count > class_nums[1] and count <= class_nums[2]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' G' + '\n')
            if count > class_nums[2] and count <= class_nums[3]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' C' + '\n')
            if count > class_nums[3]:
                f.write('exclude area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1))
            count = count + 1
    f.close()

In [ ]:
def export_data(data, export_name):
    data_vec = np.reshape(data, (len(data[1,:])*len(data[:,1])))
    
    import struct

    f=open(export_name,"wb")
    export_TL = data_vec
    export_TL.dtype
    myfmt='f'*len(export_TL)
    #  You can use 'd' for double and < or > to force endinness
    bin=struct.pack(myfmt,*export_TL)
    f.write(bin)
    f.close()

In [ ]:
s2_se_matched, s2_se_peak_basics, s2_se_truth = dump_matched_peaks(se_truth, se_data, se_pb)

In [ ]:
s2_ar_matched, s2_ar_peak_basics, s2_ar_truth = dump_matched_peaks(ar_truth, ar_data, ar_pb)

In [ ]:
len(s2_ar_matched)

In [ ]:
ar_peaks_truth = np.reshape(s2_ar_matched, [len(s2_ar_matched)])

In [ ]:
ar_peaks_truth['type'] = s2_ar_truth['type']

In [ ]:
len(s2_ar_truth[s2_ar_truth['type'] == 1])

In [ ]:
s1_ar_peaks = ar_peaks_truth[ar_peaks_truth['type'] == 1]

In [ ]:
len(s1_ar_peaks)

In [ ]:
ar_pb_truth = np.reshape(s2_ar_peak_basics, [len(s2_ar_peak_basics)])
ar_pb_truth['type'] = s2_ar_truth['type']
s1_ar_pb_truth = ar_pb_truth[ar_pb_truth['type'] == 1]

In [ ]:
len(ar_pb_truth)

In [ ]:
len(s1_ar_pb_truth)

In [ ]:
rn_s1 = rn_data[rn_data['type'] == 1]

In [ ]:
se_reshaped = np.reshape(s2_se_matched, [len(s2_se_matched)])

In [ ]:
_, se_deciles = compute_wf_and_quantiles(np.reshape(s2_se_matched, [len(s2_se_matched)]), 10)

In [ ]:
_, ar_s1_deciles = compute_wf_and_quantiles(s1_ar_peaks, 10)

In [ ]:
_, rn_s1_deciles = compute_wf_and_quantiles(rn_s1, 10)

In [ ]:
#I should probably devide by the value of the actual data for consistancy

In [ ]:
area_scaling = np.load('area_scaling.npz')['arr_0']

In [ ]:
area_scaling

In [ ]:
area_norm = np.load('area_norm.npz')['arr_0']

In [ ]:
area_norm

In [ ]:
decile_norm = np.load('deciles_scaling.npz')['arr_0']

In [ ]:
decile_norm = np.max(decile_norm)

In [ ]:
se_deciles_log = np.log(se_deciles) / decile_norm
ar_deciles_log = np.log(ar_s1_deciles) / decile_norm
rn_deciles_log = np.log(rn_s1_deciles) / decile_norm

In [ ]:
np.shape(ar_deciles_log)

In [ ]:
norm_log_decile = np.concatenate((se_deciles_log, ar_deciles_log))

In [ ]:
se_log_area = np.log10(se_reshaped['area'] + area_scaling+1) / area_norm
ar_log_area = np.log10(s1_ar_peaks['area'] + area_scaling+1) / area_norm
rn_log_area = np.log10(rn_s1['area'] + area_scaling+1) / area_norm

In [ ]:
np.shape(ar_log_area)

In [ ]:
se_aft = s2_se_peak_basics['area_fraction_top']

In [ ]:
se_aft = np.reshape(se_aft, [len(se_aft)])                    

In [ ]:
se_aft = np.where(se_aft > 0, se_aft, 0)
se_aft = np.where(se_aft < 1, se_aft, 1)

In [ ]:
ar_aft = s1_ar_pb_truth['area_fraction_top']
ar_aft = np.reshape(ar_aft, [len(ar_aft)])

In [ ]:
np.shape(ar_aft)

In [ ]:
ar_aft = np.where(ar_aft > 0, ar_aft, 0)
ar_aft = np.where(ar_aft < 1, ar_aft, 1)

In [ ]:
rn_aft = rn_pb['area_fraction_top']

In [ ]:
rn_aft = np.where(rn_aft > 0, rn_aft, 0)
rn_aft = np.where(rn_aft < 1, rn_aft, 1)

In [ ]:
se_s1_decile_area = np.concatenate((se_deciles_log, np.reshape(se_log_area, [len(se_log_area), 1])), axis = 1)

In [ ]:
np.shape(se_s1_decile_area)

In [ ]:
se_s1_decile_area_aft = np.concatenate((s1_decile_area, np.reshape(se_aft, [len(se_aft), 1])), axis = 1)

In [ ]:
np.shape(se_s1_decile_area_aft)

In [ ]:
def make_data_inputs(data, area, aft):
    decile_area = np.concatenate((data, np.reshape(area, [len(area), 1])), axis = 1)
    decile_area_aft = np.concatenate((decile_area, np.reshape(aft, [len(aft), 1])), axis = 1)
    
    return decile_area_aft

In [ ]:
ar_s1_deciles_area_aft = make_data_inputs(ar_deciles_log, ar_log_area, ar_aft) 

In [ ]:
rn_s1_deciles_area_aft = make_data_inputs(rn_deciles_log, rn_log_area, rn_aft) 

In [ ]:
np.shape(se_s1_decile_area_aft)

In [ ]:
np.shape(ar_s1_deciles_area_aft)

In [ ]:
np.shape(rn_s1_deciles_area_aft)

In [ ]:
def factors(n):    
    from functools import reduce
    
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [ ]:
len(factors(11422+2))/2

In [ ]:
102*112 #+2

In [ ]:
143*156 #+2

In [ ]:
1554*2294 #+3

In [ ]:
se_decile_area_aft_zeros = np.concatenate((se_s1_decile_area_aft, np.zeros((2,12))))

In [ ]:
ar_s1_decile_area_aft_zeros = np.concatenate((ar_s1_deciles_area_aft, np.zeros((2,12))))

In [ ]:
rn_s1_deciles_area_aft_zeros = np.concatenate((rn_s1_deciles_area_aft, np.zeros((3,12))))

In [ ]:
generate_incl_file_4('./files_for_NS/se_decile_area_aft.incl', 102, 112, [102*112, 1000000000, 1000000000, 1000000000])

In [ ]:
generate_incl_file_4('./files_for_NS/ar_decile_area_aft.incl', 143, 156, [143*156, 1000000000, 1000000000, 1000000000])

In [ ]:
generate_incl_file_4('./files_for_NS/rn_decile_area_aft.incl', 1554, 2294, [143*156, 1000000000, 1000000000, 1000000000])

In [ ]:
export_data(se_decile_area_aft_zeros , './saved_data/sim_SE_dec_area_aft.raw')

In [ ]:
export_data(ar_s1_decile_area_aft_zeros , './saved_data/sim_ar_s1_dec_area_aft.raw')

In [ ]:
export_data(rn_s1_deciles_area_aft_zeros , './saved_data/cal_rn24072_s1_dec_area_aft.raw')

In [ ]:
plt.plot(np.mean(rn_s1_deciles_area_aft_zeros, axis = 0))

In [ ]:
plt.plot(rn_s1_deciles_area_aft_zeros[0])

In [ ]:
1+1

In [ ]:
np.mean(rn_s1_deciles_area_aft_zeros)

In [ ]:
np.var(rn_s1_deciles_area_aft_zeros)

In [ ]:
ar_37_data = np.load('./sim_data/Ar37_sim_s1.npy')

In [ ]:
fig = plt.figure(figsize=(6,4))
#plt.scatter(class0_2['area'],class0_2['rise_time'], s= 0.1, marker='.',color='b', label = "Class 1")
plt.scatter(ar_37_data['area'][ar_37_data['type'] == 1],
            ar_37_data['rise_time'][ar_37_data['type'] == 1], 
            s= 1, marker='.',color='blue', label = "S1")
plt.scatter(ar_37_data['area'][ar_37_data['type'] == 2],
            ar_37_data['rise_time'][ar_37_data['type'] == 2], 
            s= 1, marker='.',color='g', label = "S2")
#plt.scatter(class3_2['length'],class3_2['rise_time'], s= 0.1, marker='.',color='orange', label = "Class 3")
#plt.scatter(class2['area'],class2['rise_time'], s= 0.1, marker='.',color='red')
plt.yscale('log')
plt.xscale('log')
plt.xlim(1,100)
plt.ylim(1,1000)
plt.xlabel('Area [PE]')#, fontsize = 30)
plt.ylabel('Rise time [ns]')#, fontsize = 30)
plt.legend(markerscale=8)#, fontsize = 24)
plt.tight_layout()
#plt.savefig('Checks/kr83_real_data_3lbl_SOM_class.pdf')
plt.show()

In [ ]:
_, sim_ar37_s1_decile = compute_wf_and_quantiles(ar_37_data, 10)

In [ ]:
sim_ar37_s1_decile = np.log(sim_ar37_s1_decile) / decile_norm

In [ ]:
sim_ar37_s1_log_area = np.log10(ar_37_data['area'] + area_scaling+1) / area_norm

In [ ]:
sim_ar37_s1_log_area = np.reshape(sim_ar37_s1_log_area, [len(sim_ar37_s1_log_area), 1])

In [ ]:
ar_37_data_aft = ar_37_data['area_fraction_top']

In [ ]:
ar_37_data_aft = np.where(ar_37_data_aft > 0, ar_37_data_aft, 0)
ar_37_data_aft = np.where(ar_37_data_aft < 1, ar_37_data_aft, 1)

In [ ]:
ar_37_data_aft = np.reshape(ar_37_data_aft, [len(ar_37_data_aft), 1])

In [ ]:
ar_sim_decile_area = np.concatenate((sim_ar37_s1_decile, sim_ar37_s1_log_area), axis = 1)

In [ ]:
np.shape(ar_sim_decile_area)

In [ ]:
ar_sim_decile_area_aft = np.concatenate((ar_sim_decile_area, ar_37_data_aft), axis= 1)

In [ ]:
ar_sim_decile_area_aft_zeros = np.concatenate((ar_sim_decile_area_aft, np.zeros((8,12))))

In [ ]:
export_data(ar_sim_decile_area_aft_zeros , './saved_data/sim_Ar37_S1_dec_area_aft.raw')

In [ ]:
# Figure out how many should be misclassified for the Ar as the ground truth is attached to it

In [ ]:
def compute_s1_boundary(parm, area):
    boundary_line = parm[0]*np.exp(-area/parm[1]) + parm[2]
    
    return boundary_line

In [ ]:
area_linspace =  np.linspace(1, 100, 200)
parm2 = np.array([100, 80, 12])

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(ar_37_data['area'][ar_37_data['type'] == 1], 
            ar_37_data['rise_time'][ar_37_data['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(ar_37_data['area'][ar_37_data['type'] == 2], 
            ar_37_data['rise_time'][ar_37_data['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=8)
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
ar_37_data.dtype